# SYMFLUENCE Tutorial 02b — Basin-Scale Workflow (Bow River at Banff, Semi-Distributed)

## Introduction

This tutorial advances from lumped to semi-distributed watershed modeling. Instead of representing the basin as a single unit, we now subdivide it into multiple connected sub-basins (GRUs) that capture spatial variability while maintaining computational efficiency.

Building on Tutorial 02a's lumped approach, semi-distributed modeling adds spatial detail through automated watershed delineation that creates multiple sub-basins, stream network topology that connects GRUs through routing, and spatially-distributed characteristics that better represent elevation gradients and heterogeneous processes.

The key configuration change is `DOMAIN_DEFINITION_METHOD: 'delineate'` with a `STREAM_THRESHOLD` parameter controlling the number of sub-basins. Smaller thresholds create more GRUs (finer spatial detail) but increase computational cost.

We continue with the **Bow River at Banff** watershed, now discretized into multiple GRUs connected by mizuRoute for explicit stream network routing. This approach improves representation of snowmelt timing, spatial climate variability, and runoff generation patterns.


# Step 1 — Configuration and data reuse

We generate a semi-distributed configuration and intelligently reuse data from Tutorial 02a where possible.

In [ ]:
# Step 1 — Semi-distributed configuration with data reuse

from pathlib import Path
import yaml
import shutil
import sys
sys.path.append(str(Path("../..").resolve()))
from symfluence import SYMFLUENCE

# Define directories
SYMFLUENCE_CODE_DIR = Path("../..").resolve()
SYMFLUENCE_DATA_DIR = Path("/Users/darrieythorsson/compHydro/test/SYMFLUENCE_data/").resolve()

# Load template
config_template = SYMFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'
with open(config_template, 'r') as f:
    config = yaml.safe_load(f)

# === Modify for semi-distributed basin ===
config['SYMFLUENCE_CODE_DIR'] = str(SYMFLUENCE_CODE_DIR)
config['SYMFLUENCE_DATA_DIR'] = str(SYMFLUENCE_DATA_DIR)
config['DOMAIN_NAME'] = 'Bow_at_Banff_semi_distributed'
config['EXPERIMENT_ID'] = 'run_1'
config['POUR_POINT_COORDS'] = '51.1722/-115.5717'

# Key changes for semi-distributed
config['DELINEATION_METHOD'] = 'stream_threshold'  # Watershed subdivision
config['DOMAIN_DEFINITION_METHOD'] = 'delineate'  # Watershed
config['STREAM_THRESHOLD'] = 5000  # Controls number of sub-basins
config['DOMAIN_DISCRETIZATION'] = 'GRUs'

config['HYDROLOGICAL_MODEL'] = 'SUMMA'
config['ROUTING_MODEL'] = 'mizuRoute'
config['MIZU_FROM_MODEL'] = 'SUMMA'


config['SETTINGS_MIZU_ROUTING_VAR'] = 'averageRoutedRunoff'
config['SETTINGS_MIZU_ROUTING_UNITS'] = 'm/s'
config['SETTINGS_MIZU_ROUTING_DT'] = '3600'

# Temporal extent
config['EXPERIMENT_TIME_START'] = '2004-01-01 01:00'
config['EXPERIMENT_TIME_END'] = '2007-12-31 23:00'
config['CALIBRATION_PERIOD'] = '2005-10-01, 2006-09-30'
config['EVALUATION_PERIOD'] = '2006-10-01, 2007-12-30'
config['SPINUP_PERIOD'] = '2004-01-01, 2005-09-30'

config['STATION_ID'] = '05BB001'
config['DOWNLOAD_WSC_DATA'] = True

config['PARAMS_TO_CALIBRATE'] = 'minStomatalResistance,cond2photo_slope,vcmax25_canopyTop,jmax25_scale,summerLAI,rootingDepth,soilStressParam,z0Canopy,windReductionParam'
config['OPTIMISATION_TARGET'] = 'streamflow'
config['ITERATIVE_OPTIMIZATION_ALGORITHM'] = 'DDS'
config['OPTIMIZATION_METRIC'] = 'KGE'
config['CALIBRATION_TIMESTEP'] = 'hourly'  

# Save configuration
config_path = SYMFLUENCE_CODE_DIR / '0_config_files' / 'config_semi_distributed.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print(f"✅ Configuration saved: {config_path}")

# === Data reuse from Tutorial 02a ===
lumped_domain = 'Bow_at_Banff_lumped'
lumped_data_dir = SYMFLUENCE_DATA_DIR / f'domain_{lumped_domain}'

def copy_with_name_adaptation(src, dst, old_name, new_name):
    """Copy directory and adapt filenames"""
    if not src.exists():
        return False
    dst.parent.mkdir(parents=True, exist_ok=True)
    if src.is_file():
        shutil.copy2(src, dst)
        return True
    shutil.copytree(src, dst, dirs_exist_ok=True)
    # Rename files containing old domain name
    for file in dst.rglob('*'):
        if file.is_file() and old_name in file.name:
            new_file = file.parent / file.name.replace(old_name, new_name)
            file.rename(new_file)
    return True

# Initialize SYMFLUENCE first
symfluence = SYMFLUENCE(config_path)
project_dir = symfluence.managers['project'].setup_project()

if lumped_data_dir.exists():
    print(f"\n📋 Reusing data from Tutorial 02a: {lumped_data_dir}")
    
    reusable_data = {
        'Elevation': lumped_data_dir / 'attributes' / 'elevation',
        'Land Cover': lumped_data_dir / 'attributes' / 'landclass',
        'Soils': lumped_data_dir / 'attributes' / 'soilclass',
        'Forcing': lumped_data_dir / 'forcing' / 'raw_data',
        'Streamflow': lumped_data_dir / 'observations' / 'streamflow'
    }
    
    for data_type, src_path in reusable_data.items():
        if src_path.exists():
            rel_path = src_path.relative_to(lumped_data_dir)
            dst_path = project_dir / rel_path
            success = copy_with_name_adaptation(src_path, dst_path, lumped_domain, config['DOMAIN_NAME'])
            if success:
                print(f"   ✅ {data_type}: Copied")
        else:
            print(f"   📋 {data_type}: Not found")
else:
    print(f"\n⚠️  No data from Tutorial 02a found. Will acquire fresh data.")

# Create pour point
pour_point_path = symfluence.managers['project'].create_pour_point()
print(f"\n✅ Project structure created at: {project_dir}")

## Step 2 — Domain definition (multi-GRU)

Delineate the watershed into multiple sub-basins using stream network analysis and create connected GRUs.

### Step 2a — Attribute check

Verify DEM availability from data reuse, or acquire fresh if needed.

In [ ]:
# Step 2a — DEM availability check
dem_path = project_dir / 'attributes' / 'elevation' / 'dem'
if not dem_path.exists() or len(list(dem_path.glob('*.tif'))) == 0:
    print("   DEM not found, acquiring geospatial attributes...")
    # If using MAF supported HPC, uncomment the line below
    # symfluence.managers['data'].acquire_attributes()
    print("✅ Geospatial attributes acquired")
else:
    print("✅ DEM available from previous workflow")

### Step 2b — Stream network delineation

Automated watershed subdivision based on stream threshold parameter.

In [ ]:
# Step 2b — Stream network delineation
watershed_path = symfluence.managers['domain'].define_domain()
print("✅ Stream network delineation complete")

### Step 2c — GRU discretization

Convert sub-basins to GRUs with routing connectivity.

In [ ]:
# Step 2c — GRU discretization
hru_path = symfluence.managers['domain'].discretize_domain()
print("✅ GRU discretization complete")

### Step 2d — Network visualization

Visualize the semi-distributed structure: sub-basins and stream network.

In [ ]:
# Step 2d — Network structure visualization

import geopandas as gpd
import matplotlib.pyplot as plt

# Load spatial products
basin_dir = project_dir / 'shapefiles' / 'river_basins'
network_dir = project_dir / 'shapefiles' / 'river_network'

basin_files = list(basin_dir.glob('*.shp'))
network_files = list(network_dir.glob('*.shp'))

if basin_files:
    basins_gdf = gpd.read_file(basin_files[0])
    print(f"Number of GRUs: {len(basins_gdf)}")
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    basins_gdf.boundary.plot(ax=ax, color='blue', linewidth=1)
    basins_gdf.plot(ax=ax, column='GRU_ID', cmap='tab20', alpha=0.5, legend=False)
    
    if network_files:
        network_gdf = gpd.read_file(network_files[0])
        network_gdf.plot(ax=ax, color='darkblue', linewidth=2, label='Stream Network')
    
    pour_point_gdf = gpd.read_file(pour_point_path)
    pour_point_gdf.plot(ax=ax, color='red', markersize=150, marker='*', label='Pour Point')
    
    ax.set_title(f'Semi-Distributed Structure\n{len(basins_gdf)} Connected GRUs', fontweight='bold')
    ax.legend()
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Basin shapefiles not found")

## Step 3 — Data preprocessing

Process forcing and observation data for multiple GRUs.

In [ ]:
# Step 3a — Streamflow observations
# If using MAF supported HPC, uncomment the line below
# symfluence.managers['data'].process_observed_data()
print("✅ Streamflow data processing complete")

In [ ]:
# Step 3b — Forcing data
# If using MAF supported HPC, uncomment the line below  
# symfluence.managers['data'].acquire_forcings()
print("✅ Forcing acquisition complete")

In [ ]:
# Step 3c — Model-agnostic preprocessing
symfluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Model-agnostic preprocessing complete")

## Step 4 — Model execution

Configure and run SUMMA-mizuRoute with multiple connected GRUs.

In [ ]:
# Step 4a — Model configuration
symfluence.managers['model'].preprocess_models()
print("✅ Semi-distributed model configuration complete")

In [ ]:
# Step 4b — Model execution
print(f"Running {config['HYDROLOGICAL_MODEL']} with {config['ROUTING_MODEL']} ({len(basins_gdf)} GRUs)...")
symfluence.managers['model'].run_models()
print("✅ Semi-distributed simulation complete")

## Step 5 — Evaluation

Compare semi-distributed results against observations.

In [ ]:
# Step 5 — Semi-distributed evaluation

import pandas as pd
import numpy as np
import xarray as xr

# --- 1) Observations (make daily & tz-naive) ---
obs_path = project_dir / "observations" / "streamflow" / "preprocessed" / f"{config['DOMAIN_NAME']}_streamflow_processed.csv"
obs_df = pd.read_csv(obs_path, parse_dates=["datetime"]).set_index("datetime").sort_index()
# If tz-aware, make tz-naive for matching
if obs_df.index.tz is not None:
    obs_df.index = obs_df.index.tz_convert(None)
# Put obs on daily mean (adjust if your obs are daily totals/instants)
obs_daily = obs_df["discharge_cms"].resample("D").mean().rename("discharge_obs")

# --- 2) Routed simulation: select the gauge segment & make daily ---
routing_dir = project_dir / "simulations" / config["EXPERIMENT_ID"] / "mizuRoute"
sim_files = list(routing_dir.glob("*.nc"))
if not sim_files:
    raise FileNotFoundError(f"No routed streamflow in: {routing_dir}")

ds = xr.open_dataset(sim_files[0])

# --- Auto-pick the segment with the highest flow ---
var = ds["IRFroutedRunoff"]

# Identify time and segment dims robustly
time_dim = next((d for d in var.dims if "time" in d.lower()), None)
if time_dim is None:
    raise RuntimeError("Could not find a time dimension in IRFroutedRunoff.")

seg_dims = [d for d in var.dims if d != time_dim]
if len(seg_dims) == 0:
    # 1D over time (no segment dimension) -> just use it
    sim_series = var
else:
    # Assume single segment dimension; if multiple, just take the first
    seg_dim = seg_dims[0]

    # Compute mean over time per segment (skip NaNs), pick the largest
    seg_mean = var.mean(time_dim, skipna=True)
    # argmax returns an index along seg_dim
    max_idx = seg_mean.argmax(dim=seg_dim)

    # Get the coordinate value (segment id) at that index
    seg_id = seg_mean[seg_dim].isel({seg_dim: max_idx}).item()

    # Select that segment’s time series
    sim_series = var.sel({seg_dim: seg_id})

    # Optional: quick sanity log
    print(f"Selected segment dim: {seg_dim}, id: {seg_id}")
    try:
        print("Segment mean flow (m^3/s):", float(seg_mean.sel({seg_dim: seg_id}).values))
    except Exception:
        pass


# Convert to pandas
sim_df = sim_series.to_series().sort_index()
# Make tz-naive if needed
if getattr(sim_df.index, "tz", None) is not None:
    sim_df.index = sim_df.index.tz_convert(None)

# Put sim on daily mean to match obs cadence (change to .sum() if routed values are volumes)
sim_daily = sim_df.resample("D").mean().rename("discharge_sim")

# --- 3) Align & join on dates we actually have in obs ---


spinup_end = (sim_daily.index.min() + pd.DateOffset(years=1)).normalize()



# Trim
obs_daily_trim = obs_daily.loc[obs_daily.index >= spinup_end]
sim_daily_trim = sim_daily.loc[sim_daily.index >= spinup_end]

# Rebuild evaluation frame
eval_df = pd.concat([obs_daily_trim.rename("discharge_obs"),
                     sim_daily_trim.rename("discharge_sim")], axis=1) \
             .dropna(subset=["discharge_obs", "discharge_sim"]) \
             .sort_index()

# Short-circuit if empty
if eval_df.empty:
    raise RuntimeError("After spinup trimming there is no overlap. Check your spinup_end rule/time ranges.")

obs_valid = eval_df["discharge_obs"]
sim_valid = eval_df["discharge_sim"]

# --- 4) Robust metrics ---
def _safe_div(a, b):
    return np.nan if b == 0 or np.isnan(b) else a / b

def nse(obs, sim):
    num = np.nansum((obs - sim) ** 2)
    den = np.nansum((obs - np.nanmean(obs)) ** 2)
    return float(1 - _safe_div(num, den))

def kge(obs, sim):
    r = np.corrcoef(obs, sim)[0, 1]
    alpha = np.nanstd(sim) / np.nanstd(obs)
    beta = np.nanmean(sim) / np.nanmean(obs)
    return float(1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2))

def pbias(obs, sim):
    num = np.nansum(sim - obs)
    den = np.nansum(obs)
    return float(100 * _safe_div(num, den))

nse_val = np.round(nse(obs_valid, sim_valid), 3)
kge_val = np.round(kge(obs_valid, sim_valid), 3)
pbias_val = np.round(pbias(obs_valid, sim_valid), 1)

print(f"Performance Metrics:")
print(f"  NSE: {nse_val}")
print(f"  KGE: {kge_val}")
print(f"  PBIAS: {pbias_val}%")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series
axes[0, 0].plot(obs_valid.index, obs_valid.values, 'b-', label='Observed', linewidth=1.2, alpha=0.7)
axes[0, 0].plot(sim_valid.index, sim_valid.values, 'r-', label=f'Semi-Distributed ({len(basins_gdf)} GRUs)', 
                linewidth=1.2, alpha=0.7)
axes[0, 0].set_ylabel('Discharge (m³/s)')
axes[0, 0].set_title('Semi-Distributed Streamflow')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].text(0.02, 0.95, f"NSE: {nse_val}\nKGE: {kge_val}\nBias: {pbias_val}%\nGRUs: {len(basins_gdf)}",
                transform=axes[0, 0].transAxes, verticalalignment='top',
                bbox=dict(facecolor='white', alpha=0.8), fontsize=9)

# Scatter
axes[0, 1].scatter(obs_valid, sim_valid, alpha=0.5, s=10, c='green')
max_val = max(obs_valid.max(), sim_valid.max())
axes[0, 1].plot([0, max_val], [0, max_val], 'k--', alpha=0.5)
axes[0, 1].set_xlabel('Observed (m³/s)')
axes[0, 1].set_ylabel('Simulated (m³/s)')
axes[0, 1].set_title('Observed vs Simulated')
axes[0, 1].grid(True, alpha=0.3)

# Monthly climatology
monthly_obs = obs_valid.groupby(obs_valid.index.month).mean()
monthly_sim = sim_valid.groupby(sim_valid.index.month).mean()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
axes[1, 0].plot(monthly_obs.index, monthly_obs.values, 'b-o', label='Observed', markersize=6)
axes[1, 0].plot(monthly_sim.index, monthly_sim.values, 'r-o', label='Simulated', markersize=6)
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].set_xticklabels(month_names)
axes[1, 0].set_ylabel('Mean Discharge (m³/s)')
axes[1, 0].set_title('Seasonal Flow Regime')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Flow duration curve
obs_sorted = obs_valid.sort_values(ascending=False)
sim_sorted = sim_valid.sort_values(ascending=False)
obs_ranks = np.arange(1., len(obs_sorted) + 1) / len(obs_sorted) * 100
sim_ranks = np.arange(1., len(sim_sorted) + 1) / len(sim_sorted) * 100
axes[1, 1].semilogy(obs_ranks, obs_sorted, 'b-', label='Observed', linewidth=2)
axes[1, 1].semilogy(sim_ranks, sim_sorted, 'r-', label='Simulated', linewidth=2)
axes[1, 1].set_xlabel('Exceedance Probability (%)')
axes[1, 1].set_ylabel('Discharge (m³/s)')
axes[1, 1].set_title('Flow Duration Curve')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle(f'Semi-Distributed Evaluation — {config["DOMAIN_NAME"]} ({len(basins_gdf)} GRUs)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Semi-distributed evaluation complete")

# Step 5b — Run calibration 



In [ ]:
results_file = symfluence.managers['optimization'].calibrate_model()  
print("Calibration results file:", results_file)